# Extract hospital performance for pathway model

## Aims

* Extract and save hospital performance for pathway simulation model
* Create breakdowns by weekend/weekday/day/night

## Import libraries

In [3]:
import numpy as np
import pandas as pd

## Load data

* Load data
* Restrict data to fields necessary for pathway extraction
* Remove in-hospital admissions

In [4]:
# Load data
data_loaded = pd.read_csv(
    './data/SAMueL ssnap extract v2.csv', low_memory=False)

# Number of years data covers
# January 2016 to December 2021
data_years = 6.0

In [5]:
# Restrict fields
used_fields = [
    'TeamName',
    'S1Gender',
    'OnsettoArrivalMinutes',
    'S1OnsetTimeType',
    'ArrivaltoBrainImagingMinutes',
    'S2StrokeType',
    'S2Thrombolysis',                  # Yes/No thrombolysis
    'ArrivaltoThrombolysisMinutes',    # thrombolysis time
    'ArrivaltoArterialPunctureMinutes' # thrombectomy time
]

data_loaded = data_loaded[used_fields]

## Extract hospital performance

█▓▒░

In [107]:
def analyse_by_team(stroke_team: str, group_df: pd.DataFrame):
    """
    █▓▒░
    
    With each step, whittle down the full group of patients in the
    following way. In the example, the sizes of blocks are arbitrary. 
    
    Key:
    ░ - patients still in the subgroup
    ▒ - patients rejected from the subgroup at this step
    █ - patients rejected from the subgroup in previous steps
    
    ▏Start: Full group                                                ▕
    ░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░
    ▏-------------------------All patients----------------------------▕
    ▏                                                                 ▕
    ▏Mask 1: Is onset time known?                                     ▕
    ░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒
    ▏--------------------Yes----------------------▏---------No--------▕
    ▏                                             ▏                   ▕
    ▏Mask 2: Is arrival within x hours?           ▏                   ▕
    ░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░▒▒▒▒▒▒▒▒▒▒▒█████████████████████
    ▏---------------Yes----------------▏----No----▏------Rejected-----▕
    ▏                                  ▏          ▏                   ▕
    ▏Mask 3: Is scan within x hours of arrival?   ▏                   ▕
    ░░░░░░░░░░░░░░░░░░░░░░░░░░░░▒▒▒▒▒▒▒████████████████████████████████
    ▏------------Yes------------▏--No--▏-----------Rejected-----------▕
    ▏                           ▏      ▏                              ▕
    ▏Mask 4: Is scan within x hours of onset?                         ▕
    ░░░░░░░░░░░░░░░░░░░░░░░▒▒▒▒▒███████████████████████████████████████
    ▏----------Yes---------▏-No-▏---------------Rejected--------------▕
    ▏                      ▏    ▏                                     ▕
    ▏Mask 5: Is there enough time left for thrombolysis/thrombectomy? ▕
    ░░░░░░░░░░░░░░░░▒▒▒▒▒▒▒████████████████████████████████████████████
    ▏------Yes------▏--No--▏------------------Rejected----------------▕
    ▏               ▏      ▏                                          ▕
    ▏Mask 6: Did the patient receive thrombolysis/thrombectomy?       ▕
    ░░░░░░░░░░░▒▒▒▒▒███████████████████████████████████████████████████
    ▏----Yes---▏-No-▏---------------------Rejected--------------------▕


    Patient proportions measured:    
    +--------------------------------+--------------------------------+
    | Proportion                     | Measure                        |
    +--------------------------------+--------------------------------+
    | Thrombolysis rate or           | Total number treated such      |
    | thrombectomy rate              | divided by all patients.       |
    | Onset known                    | "Yes" to Mask 1 divided by     |
    |                                | all patients.                  |
    | Known arrival within x hours   | "Yes" to Mask 2 divided by     |
    |                                | "Yes" to Mask 1.               |
    | Scan within x hours of arrival | "Yes" to Mask 3 divided by     |
    |                                | "Yes" to Mask 2.               |
    | Scan within x hours of onset   | "Yes" to Mask 4 divided by     |
    |                                | "Yes" to Mask 3.               |
    | Enough time left for treatment | "Yes" to Mask 5 divided by     |
    |                                | "Yes" to Mask 4.               |
    | "Chosen" for thrombolysis      | "Yes" to Mask 6 divided by     |
    | or for thrombectomy            | "Yes" to Mask 5.               |
    +--------------------------------+--------------------------------+
    The "proportion chosen for thrombolysis" is a different measure
    from the "thrombolysis rate", which is the proportion of all of the
    patients at the start who were given thrombolysis. It is possible
    some patients received thrombolysis in real life but that by this
    process they were rejected before Mask 6.

    The log-normal mean and standard deviation (mu and sigma) are taken
    for the groups of patients who answer "Yes" to everything up to and
    including particular steps.
    +---------------------------------+-------------------------------+
    | Subgroup who answer "yes" at... | Log-normal distribution       |
    +---------------------------------+-------------------------------+
    |                          Mask 2 | Onset to arrival time         |
    |                          Mask 3 | Arrival to scan time          |
    |                          Mask 4 | Scan to needle or             |
    |                                 | scan to puncture time         |
    +---------------------------------+-------------------------------+
    
    The same process is used to determine stats about thrombectomy,
    with the following changes:
    - the time limit is 8 hours instead of 4 hours.
    - Mask 4 determines the scan to puncture time.
    - if fewer than 10 patients receive thrombectomy per year,
      the mu and sigma for scan to puncture time are discarded. ------------------------- is this implemented yet?
      
    """
    def calculate_more_times_for_dataframe(group_df):
        # Scan to treatment
        group_df['scan_to_needle_mins'] = (
            group_df['ArrivaltoThrombolysisMinutes'] -
            group_df['ArrivaltoBrainImagingMinutes']
        )
        group_df['scan_to_puncture_mins'] = (
            group_df['ArrivaltoArterialPunctureMinutes'] -
            group_df['ArrivaltoBrainImagingMinutes']
        )
        
        # Replace any zero scan to treatment times with 1 (for log?)
        scan_to_needle_mins = group_df['scan_to_needle_mins'].copy()
        scan_to_needle_mins[scan_to_needle_mins == 0] = 1
        group_df['scan_to_needle_mins'] = scan_to_needle_mins
                    
        scan_to_puncture_mins = group_df['scan_to_puncture_mins'].copy()
        scan_to_puncture_mins[scan_to_puncture_mins == 0] = 1
        group_df['scan_to_puncture_mins'] = scan_to_puncture_mins
            
        
        # Time left after scan for thrombolysis...
        group_df['time_left_for_ivt_after_scan_mins'] = np.maximum((
            allowed_onset_to_needle_time_mins -
            (group_df['OnsettoArrivalMinutes'] + 
              group_df['ArrivaltoBrainImagingMinutes'])
            ), -0.0)
        # ... and thrombectomy:
        group_df['time_left_for_mt_after_scan_mins'] = np.maximum((
            allowed_onset_to_puncture_time_mins -
            (group_df['OnsettoArrivalMinutes'] + 
              group_df['ArrivaltoBrainImagingMinutes'])
            ), -0.0)
        return group_df
    
    
    def create_masks(
            time_left_after_scan_mins: np.array,
            time_to_treatment_mins: np.array,        
            limit_mins: float=np.inf,
            ):
        """
        eh?
        """
        # Create masks.
        # 1: Onset time known
        mask1 = group_df[
            'S1OnsetTimeType'].apply(lambda x: x in ['P', 'BE'])
        # 2: Mask 1 and onset to arrival on time
        mask2 = ((mask1 == True) & 
                 (group_df['OnsettoArrivalMinutes'] <= limit_mins))
        # 3: Mask 2 and arrival to scan on time
        mask3 = (
            (mask2 == True) & 
            (group_df['ArrivaltoBrainImagingMinutes'] <= limit_mins)
            )
        # 4: Mask 3 and onset to scan on time
        mask4 = (
            (mask3 == True) &
            ((group_df['OnsettoArrivalMinutes'] + 
              group_df['ArrivaltoBrainImagingMinutes']) <= limit_mins)
            )
        # 5: Mask 4 and enough time to treat
        mask5 = (
            (mask4 == True) &
            (time_left_after_scan_mins >= minutes_left)  # Should "enough time" instead measure whether onset to needle is below the limit?
            )        
        # 6: Mask 5 and treated
        mask6 = ((mask5 == True) & 
                 (time_to_treatment_mins >= 0))
        
        masks_dict = dict(
            mask1_all_onset_known=mask1,
            mask2_mask1_and_onset_to_arrival_on_time=mask2,
            mask3_mask2_and_arrival_to_scan_on_time=mask3, 
            mask4_mask3_and_onset_to_scan_on_time=mask4, 
            mask5_mask4_and_enough_time_to_treat=mask5, 
            mask6_mask5_and_treated=mask6
            )
        return masks_dict

    
    def calculate_proportions(masks_dict):
        """
        Find proportions of patients who answer True to each mask
        out of those who answered True to the previous mask.
        """
        # Store results in here:
        proportions_dict = dict()
        # Get the mask names assuming that the masks are stored in
        # the dictionary in order.
        mask_names = list(masks_dict.keys())
        for j, mask_name_now in enumerate(mask_names):
            mask_now = masks_dict[mask_name_now]
            if j > 0:
                mask_before = masks_dict[mask_names[j-1]]
                mask_name_before = f'mask{j}'
            else:
                # All patients answer True.
                mask_before = np.full(len(mask_now), 1)
                mask_name_before = 'all'
            
            # Proportion is Yes to Mask now / Yes to Mask before.
            proportion = (np.sum(mask_now) / np.sum(mask_before)
                          if np.sum(mask_before) > 0 else np.NaN)
            # Store result with a similar name to the original mask.
            # Replace the initial "maskX_" with "proportionX_":
            
            
            proportion_name = (
                f'proportion{j+1}_of_{mask_name_before}_with'
            )
            # Remove previous mask name and "_and" from the string:
            p = '_'.join(mask_name_now[6:].split(mask_name_before)[-1:])
            p = '_and'.join(p.split('_and')[-1:])
            proportion_name += p
            proportions_dict[proportion_name] = proportion
        return proportions_dict

    
    def calculate_lognorm_parameters(group_df, input_dicts):
        """ 
        eh?
        """
        results_dict = dict()
        for d in input_dicts:
            times = np.log(group_df[d['mask']][d['column']])
            results_dict['lognorm_mu_' + d['label']] = times.mean()
            results_dict['lognorm_sigma_' + d['label']] = times.std()
        return results_dict
    

    # Set up allowed time and over-run for thrombolysis...
    allowed_onset_to_needle_time_mins = 270  # 4h 30m
    allowed_overrun_for_slow_scan_to_needle_mins = 15
    # ... and for thrombectomy
    allowed_onset_to_puncture_time_mins = 8*60  # --------------------------------- need to check for a reaonsable number here
    allowed_overrun_for_slow_scan_to_puncture_mins = 15
    minutes_left = 15.0
    
    # Limit for comparing conditions (e.g. is onset to arrival within
    # 4hrs?). Separate limits for IVT and MT:
    limit_ivt_mins = 4*60
    limit_mt_mins = 8*60  # ################################################# look up sensible value


    # Record admission numbers
    admissions = group_df.shape[0]

    # Calculate more times from the existing data:
    group_df = calculate_more_times_for_dataframe(group_df)

    # Find the proportion of the whole cohort that receives
    # treatment.
    proportion_all_ivt = (group_df['S2Thrombolysis'] == 'Y').mean()
    proportion_all_mt = (
        group_df['ArrivaltoArterialPunctureMinutes'] >= 0).mean()
    proportion_mt_also_receiving_ivt = (
        np.sum((group_df['S2Thrombolysis'] == 'Y') & 
               (group_df['ArrivaltoArterialPunctureMinutes'] >= 0)) /
        np.sum(group_df['ArrivaltoArterialPunctureMinutes'] >= 0)
        ) if np.sum(group_df['ArrivaltoArterialPunctureMinutes'] >= 0) > 0 else np.NaN

    # ----- Thrombolysis -----
    # Masks of patients who answer True to each step:
    masks_dict_ivt = create_masks(
        group_df['time_left_for_ivt_after_scan_mins'],
        group_df['ArrivaltoThrombolysisMinutes'],
        limit_ivt_mins
        )
    # Proportion of patients in each mask:
    proportions_dict_ivt = calculate_proportions(masks_dict_ivt)
    # Record the mu and sigma for certain times and subgroups.
    # Set up with these dictionaries:
    dicts_ivt = [
        dict(label = 'onset_arrival_mins',
             mask = masks_dict_ivt[
                'mask2_mask1_and_onset_to_arrival_on_time'],
             column = 'OnsettoArrivalMinutes'
             ),
        dict(label = 'arrival_scan_arrival_mins',
             mask = masks_dict_ivt[
                'mask3_mask2_and_arrival_to_scan_on_time'],
             column = 'ArrivaltoBrainImagingMinutes'
             ),
        dict(label = 'scan_needle_mins',
             mask = masks_dict_ivt['mask6_mask5_and_treated'],
             column = 'scan_to_needle_mins'
             )
        ]
    lognorm_dict_ivt = calculate_lognorm_parameters(group_df, dicts_ivt)


    # ----- Thrombectomy -----
    # Masks of patients who answer True to each step:
    masks_dict_mt = create_masks(
        group_df['time_left_for_mt_after_scan_mins'],
        group_df['ArrivaltoArterialPunctureMinutes'],
        limit_mt_mins
        )
    # Proportion of patients in each mask:
    proportions_dict_mt = calculate_proportions(masks_dict_mt)
    # Record the mu and sigma for certain times and subgroups.
    # Set up with these dictionaries:
    dicts_mt = [
        dict(label = 'onset_arrival_mins',
             mask = masks_dict_mt[
                'mask2_mask1_and_onset_to_arrival_on_time'],
             column = 'OnsettoArrivalMinutes'
             ),
        dict(label = 'arrival_scan_arrival_mins',
             mask = masks_dict_mt[
                'mask3_mask2_and_arrival_to_scan_on_time'],
             column = 'ArrivaltoBrainImagingMinutes'
             ),
        dict(label = 'scan_puncture_mins',
             mask = masks_dict_mt['mask6_mask5_and_treated'],
             column = 'scan_to_puncture_mins'
             )
        ]
    lognorm_dict_mt = calculate_lognorm_parameters(group_df, dicts_mt)

    # ----- Combine results -----
    performance_dict = dict()
    performance_dict['stroke_team'] = stroke_team
    performance_dict['admissions'] = admissions
    performance_dict['proportion_of_all_with_ivt'] = proportion_all_ivt
    performance_dict['proportion_of_all_with_mt'] = proportion_all_mt
    performance_dict['proportion_of_mt_with_ivt'] = \
        proportion_mt_also_receiving_ivt

    # Take these dictionaries from earlier...
    dicts_to_combine = [proportions_dict_ivt, lognorm_dict_ivt,
                        proportions_dict_mt, lognorm_dict_mt]
    # ... and merge them into the new results dictionary.
    for i, d in enumerate(dicts_to_combine):
        # Add extra label to prevent repeat keys in the combined dict.
        extra_label = '_ivt' if i < 2 else '_mt'
        keys = list(d.keys())
        for key in keys:
            performance_dict[key + extra_label] = d[key]

    return performance_dict

In [108]:
# Copy data
data = data_loaded.copy()
# Split data by stroke team
groups = data.groupby('TeamName') # creates a new object of groups of data

# Store each stroke team's results in this list:
list_of_series = []
for stroke_team, group_df in groups: # each group has an index + dataframe of data
    # Main results function:
    group_dict = analyse_by_team(stroke_team, group_df)
    # Update admissions, average over the full number of years:
    group_dict['admissions'] = group_dict['admissions'] / data_years
    # Convert output dict into a pandas Series:
    group_series = pd.Series(data=group_dict.values(),
                             index=group_dict.keys())
    list_of_series.append(group_series)
    
# Combine all results into one dataframe:
df_all = pd.concat(list_of_series, axis=1)

In [118]:
# Limit to hosp with > 100 admissions/year and >10 thrombolysis in total and >10 thrombectomy in total
admissions = df_all.loc['admissions']
thrombolysed = admissions * df_all.loc['proportion_of_all_with_ivt']
thrombectomy = admissions * df_all.loc['proportion_of_all_with_mt']
mask = list(np.where((
        (admissions >= (100 / data_years)) &
        (thrombolysed >= (10.0 / data_years)) &
        (thrombectomy >= (10.0 / data_years))
       ).values == True)[0])

df = df_all[mask].T

# Save
df.to_csv('data/hospital_performance_thrombectomy.csv', index=False)

# Show data for five hopsitals
df.head().T

[0, 1, 2, 5, 6, 7, 9, 10, 12, 13, 16, 18, 19, 20, 21, 23, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 44, 46, 47, 48, 49, 50, 51, 55, 61, 63, 64, 67, 68, 69, 70, 71, 72, 73, 74, 75, 77, 78, 79, 80, 81, 82, 83, 84, 85, 87, 89, 90, 91, 93, 95, 96, 98, 99, 100, 101, 102, 103, 107, 108, 110, 111, 113, 116, 117, 118, 119]


,0,1,2,5,6
stroke_team,Addenbrooke's Hospital,Basildon University Hospital,Blackpool Victoria Hospital,Broomfield Hospital,Calderdale Royal Hospital
admissions,602.166667,486.5,485.833333,452.166667,634.666667
proportion_of_all_with_ivt,0.149184,0.132237,0.091938,0.104681,0.135504
proportion_of_all_with_mt,0.026571,0.01199,0.01235,0.006635,0.00604
proportion_of_mt_with_ivt,0.520833,0.6,0.444444,0.555556,0.73913
proportion1_of_all_with_onset_known_ivt,0.590645,0.648167,0.44837,0.558791,0.546218
proportion2_of_mask1_with_onset_to_arrival_on_time_ivt,0.679007,0.57241,0.693191,0.605541,0.645673
proportion3_of_mask2_with_arrival_to_scan_on_time_ivt,0.94893,0.958449,0.941501,0.991285,0.946389
proportion4_of_mask3_with_onset_to_scan_on_time_ivt,0.845818,0.888247,0.888628,0.915385,0.929976
proportion5_of_mask4_with_enough_time_to_treat_ivt,1.0,1.0,1.0,1.0,1.0
